In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss


In [ ]:
df = pd.read_pickle("/Users/simongutwein/mnt/qb/baumgartner/sgutwein84/test_case_results.pkl")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df)


In [ ]:
entities = ["pt", "lt", "mt", "ht", "nt"]

for enti in entities:

    df = pd.read_pickle("/Users/simongutwein/mnt/qb/baumgartner/sgutwein84/test_case_results.pkl")
    df_tmp = df[df["plan"].str.contains(enti)]
    df_tmp = df_tmp[df["model"].str.contains("prost")]

    #print(enti)
    #print(df_tmp.head())
    print(enti, f"n={df_tmp.count()[0]}",np.round(df_tmp["gamma"].mean(), 2), "±", np.round(df_tmp["gamma"].std(), 2), "(", np.round(df_tmp["gamma"].median(), 2), ")")


In [ ]:
df_train = pd.read_excel("/Users/simongutwein/home/baumgartner/sgutwein84/container/segments.xlsx", index_col=0)
df_test = pd.read_excel("/Users/simongutwein/home/baumgartner/sgutwein84/container/segments_tests.xlsx", index_col=0)

In [ ]:
df_train['fieldsize'] = df_train['fieldsize'].map(lambda x: x.lstrip('tensor(').rstrip(')'))
df_test['fieldsize'] = df_test['fieldsize'].map(lambda x: x.lstrip('tensor(').rstrip(')'))

In [ ]:
cases = ["p", "h", "l", "m", "n"]
cases_num = [45,15,15,15,15]


In [ ]:
np.array(df_train[df_train['test_case'].str.match("pt")]["value"]).astype(float).mean()


In [ ]:
infos = []
for case, num in zip(cases, cases_num):

    if case == "p":
        
        infos.append({
            "case": case,
            "cases_num": num,
            "train": df_train[df_train['test_case'].str.match(case)].count()[0],
            "test": df_test[df_test['test_case'].str.match(case)].count()[0],
            "mean_train": np.array(df_train[df_train['test_case'].str.match(case)]["fieldsize"]).astype(float).mean(),
            "std_train": np.array(df_train[df_train['test_case'].str.match(case)]["fieldsize"]).astype(float).std(),
            "mean_test": np.array(df_test[df_test['test_case'].str.match(case)]["fieldsize"]).astype(float).mean(),
            "std_test": np.array(df_test[df_test['test_case'].str.match(case)]["fieldsize"]).astype(float).std(),
        })
    else:
        infos.append({
            "case": case,
            "cases_num": num,
            "train": df_train[df_train['test_case'].str.match(case)].count()[0],
            "test": df_test[df_test['test_case'].str.match(case)].count()[0],
            "mean_train": np.array(df_train[df_train['test_case'].str.match(case)]["fieldsize"]).astype(float).mean(),
            "mean_test": np.array(df_test[df_test['test_case'].str.match(case)]["fieldsize"]).astype(float).mean(),
            "std_train": np.array(df_train[df_train['test_case'].str.match(case)]["fieldsize"]).astype(float).std(),
            "std_test": np.array(df_test[df_test['test_case'].str.match(case)]["fieldsize"]).astype(float).std(),
        })

df = pd.DataFrame(infos)
df


